In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()

cursor.execute("SELECT * FROM keyword")
raw_keyword = cursor.fetchall()
for row in raw_keyword:
    # penyiapan scraping
    keyword = row[0]
    r = Request("https://www.jobstreet.co.id/id/job-search/" + keyword + "-jobs/", headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
    print(r)
    response = urlopen(r).read()
    print(type(response))

    soup = BeautifulSoup(response, "lxml")
    print(type(soup))

    # inisialisasi array untuk menampilkan hasil
    title_lowongan = []
    nama_perusahaan = []
    lokasi_perusahaan = []
    link_lowongan = []
    deskripsi_lowongan = []

    jobList = soup.find_all("div","FYwKg _20Cd9_0 _1GAuD _3MPd_ _3ftyQ _1lyEa")
    i = 1
    for p in jobList:
        lowongan = p.find('div', "FYwKg _2j8fZ_0 sIMFL_0 _1JtWu_0").get_text()
        # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
        try:
            perusahaan = p.find('span', "FYwKg _2Bz3E C6ZIU_0 _6ufcS_0 _2DNlq_0 _29m7__0").get_text()
        except:
            perusahaan = "Perusahaan Dirahasiakan"
        lokasi = p.find('span', "FYwKg _2Bz3E C6ZIU_0 _1_nER_0 _2DNlq_0 sQuda_0").get_text()
        link = "https://www.jobstreet.co.id"+p.find('a').get('href')

        cursor.execute("SELECT * FROM jobstreet WHERE link_lowongan = '" + link + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            print("Data exists")
        else:
            #   scrape detail sesuai link
            r2 = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
            print(r2)
            response2 = urlopen(r2).read()
            print(type(response2))
            
            soup2 = BeautifulSoup(response2, "html.parser")
            print(type(soup2))
            raw_deskripsi = soup2.find('div', "FYwKg d7v3r _3BZ6E_0 _2FwxQ_0")
            # increment deskripsi
            deskripsi = ""
            for string in raw_deskripsi.strings:
                new_string  = deskripsi + " " + string
                deskripsi = new_string

            # menghilangkan Apostrophe mark
            # deskripsi.replace("'","")

            cursor.execute(
                "INSERT INTO jobstreet(title_lowongan, nama_perusahaan, lokasi_perusahaan, deskripsi_lowongan, link_lowongan)"
                "VALUES ('"+ lowongan.replace("'", "").replace('"', '') +"', '"+ perusahaan.replace("'", "").replace('"', '') +"', '"+ lokasi.replace("'", "").replace('"', '') +"', '"+ deskripsi.replace("'", "").replace('"', '') +"', '"+ link +"')"
                )
            print("Data has been added")
    
    title_lowongan.append(lowongan)
    nama_perusahaan.append(perusahaan)
    lokasi_perusahaan.append(lokasi)
    deskripsi_lowongan.append(deskripsi)
    link_lowongan.append(link)

jobList_dict ={'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'deskripsi':deskripsi_lowongan, 'link':link_lowongan}
df = pd.DataFrame(jobList_dict,columns = ['lowongan','perusahaan','lokasi', 'deskripsi', 'link'])

db.commit()

df.sort_values('lowongan',ascending=True)

t.Request object at 0x0000029837CE57C0>
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added
<class 'bytes'>
<class 'bs4.BeautifulSoup'>
Data has been added


HTTPError: HTTP Error 429: Too Many Requests